In [2]:
import mediapipe as mp
import cv2
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Function to extract landmarks
def get_pose_landmarks(image):
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.pose_landmarks:
        return np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark])
    return None

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    # a, b, c are points [x, y, z]
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    
    # Cosine of the angle using dot product formula
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    
    # Ensure cosine_angle stays in the range [-1, 1] to avoid numerical errors
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    
    # Calculate the angle in degrees
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# Function to extract joint angles from landmarks
def get_joint_angles(landmarks):
    if landmarks is None:
        return None
    
    # Define key joints for comparison
    angles = {}
    
    # Example: Elbow angle (Shoulder-Elbow-Wrist)
    angles['left_elbow'] = calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                           landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                           landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])

    angles['right_elbow'] = calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])

    # Knee angle (Hip-Knee-Ankle)
    angles['left_knee'] = calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    angles['right_knee'] = calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                           landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    # Shoulder angle (Hip-Shoulder-Elbow)
    angles['left_shoulder'] = calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                              landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])

    angles['right_shoulder'] = calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                               landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    return angles

# Function to calculate the difference between two sets of joint angles
def compare_angles(angles1, angles2):
    if angles1 is None or angles2 is None:
        return float('inf')  # Return large value if one of the sets is missing
    
    # Calculate the absolute difference for each angle
    total_difference = 0
    for key in angles1:
        total_difference += abs(angles1[key] - angles2[key])
    
    return total_difference

# Load two images to compare
image1 = cv2.imread('pose3.jpg')
image2 = cv2.imread('pose1.jpg')

# Get landmarks for each pose
landmarks1 = get_pose_landmarks(image1)
landmarks2 = get_pose_landmarks(image2)

# Get joint angles for each pose
angles1 = get_joint_angles(landmarks1)
angles2 = get_joint_angles(landmarks2)

# Compare joint angles
angle_difference = compare_angles(angles1, angles2)

# Set a threshold to classify as similar
threshold = 30.0  # Adjust this based on testing
if angle_difference < threshold:
    print("Poses are similar")
else:
    print("Poses are different")

print(f"Angle difference: {angle_difference}")
print(f"Angle-1: {angles1}")
print(f"Angle -2: {angles2}")


Poses are similar
Angle difference: 14.9864614472785
Angle-1: {'left_elbow': 111.1781221515685, 'right_elbow': 117.69917145447367, 'left_knee': 44.83271621777342, 'right_knee': 40.39754822228448, 'left_shoulder': 88.1385800792194, 'right_shoulder': 101.8983743376046}
Angle -2: {'left_elbow': 112.4173565778314, 'right_elbow': 120.46038236837879, 'left_knee': 37.81285022178735, 'right_knee': 39.40597649804899, 'left_shoulder': 90.53360231508799, 'right_shoulder': 102.47793048862493}
